In [5]:
import sys
import os
sys.path.append("../")

from data_processors.dataset import load_as_hf_dataset
from config_utils.load_config import load_params_from_yaml, DataParamsSchema


In [6]:
dataset_params = load_params_from_yaml("dataset_params.yaml", DataParamsSchema)
dataset_params.data_params

DataParams(tokenizer_name='bert-base-uncased', train_data_path='data/processed/train.csv', validate_data_path='data/processed/valid.csv', test_data_path='data/processed/test.csv', subreddit1='music', subreddit2='gaming', mask_prob=0.15)

In [13]:
from pathlib import Path
root_dir = Path().resolve().parents[0]

In [14]:
from datasets import Dataset
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained(dataset_params.data_params.tokenizer_name)
dataset = load_as_hf_dataset(root_dir / dataset_params.load_params.raw_data_path)

In [18]:
dataset[:3]

{'title': ['The one feature the iPad is really missing.',
  "Dear Sydney Reddit'ers, Would you like any changes made to the style of this subreddit?",
  'I skipped bail, ran away, and never got caught. AM(A)A.'],
 'body': ["I don't care about the lack of camera. I never use the one on my MacBook, and even if I did the angle would be terrible on the iPad.\n\nI don't care if third party apps can't run in the background. I don't listen to streaming music.\n\nI don't care that the App Store is a closed system. I can jailbreak for myself and I think the closed system works better for most users.\n\nThe one feature I want is User Accounts and a Guest Account. If this device is meant to be a coffee table computer, it needs to be able to accomadate multiple users.",
  'I was going to subtly edit the style of the Sydney subreddit but then I found this post and realised that people have very strong opinions about how their reddit should look. \n\n\n\nSo before I make any changes do you have any 

In [20]:
text = dataset["body"][0]
text

"I don't care about the lack of camera. I never use the one on my MacBook, and even if I did the angle would be terrible on the iPad.\n\nI don't care if third party apps can't run in the background. I don't listen to streaming music.\n\nI don't care that the App Store is a closed system. I can jailbreak for myself and I think the closed system works better for most users.\n\nThe one feature I want is User Accounts and a Guest Account. If this device is meant to be a coffee table computer, it needs to be able to accomadate multiple users."

In [21]:
inputs = tokenizer(
    text, 
    return_tensors='pt', 
    max_length=512, 
    truncation=True, 
    padding='max_length')

In [29]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [28]:
tokenizer.special_tokens_map["mask_token"]

'[MASK]'

In [30]:
inputs['input_ids'].shape

torch.Size([1, 512])